In [1]:
import pandas as pd
import datetime as dt
import math as mt
import sklearn as skl
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
file_x_train = './data/X_train.csv'
file_y_train = './data/y_train.csv'
file_x_test = './data/X_test.csv'

In [3]:
X_train_data = pd.read_csv(file_x_train, sep=',')
Y_train_data = pd.read_csv(file_y_train, sep=',')
X_test_data = pd.read_csv(file_x_test, sep=',')

In [4]:
X_train_data=X_train_data.drop(['Unnamed: 9'],axis=1)

def f(x):
    x=x.split(':')
    x[0]=x[0].split('T')[-1]
    x=60*int(x[0])+int(x[1])
    return(x)

In [5]:
def traitementnan(vx):
    x=vx.copy()
    l=x.shape[0]
    S_point=0
    temp=0
    val=0
    for i in range(l):
        if (not (x[i]!=x[i])):
            temp=i
            val=x[i]
            for j in range(S_point,temp):
                x[j]=val
            S_point=i
    if (x[l-1]!=x[l-1]):
        for i in range(S_point,l):
            x[i]=val
    return x

In [6]:
def traitementallcolumn(X):
    for c in X.columns.tolist():
        X[c]=traitementnan(X[c])
    return X

In [7]:
def clean_na(X_train_data,Y_train_data):
    Y_train_data=Y_train_data.drop(['time_step'],axis=1)
    temp = pd.concat([X_train_data, Y_train_data],axis=1)
    temp['time_step']=temp['time_step'].apply(f)


    temp=temp.dropna(axis=0)
    
    X = temp.drop(['washing_machine','fridge_freezer','TV','kettle'],axis=1)
    Y = temp.drop(['consumption','visibility','temperature','humidity','humidex','windchill','wind','pressure'],axis=1)
    return(X, Y)

In [8]:
def traitement_outliers(X_train_data,Y_train_data,threshold=3):
    m = np.mean(X_train_data['consumption'])
    std = np.std(X_train_data['consumption'])
    X_train_clear = X_train_data[X_train_data["consumption"] < threshold*std + m]
    Y_train_clear = Y_train_data[X_train_data["consumption"] < threshold*std + m]
    return X_train_clear,Y_train_clear

def normalisation(X,Y):
    Xstock=[X['consumption'].mean(),X['consumption'].std()]
    Ystock=[[Y['washing_machine'].mean(),Y['washing_machine'].std()],
           [Y['fridge_freezer'].mean(),Y['fridge_freezer'].std()],
           [Y['TV'].mean(),Y['TV'].std()],
           [Y['kettle'].mean(),Y['kettle'].std()]]
    
    
    X['consumption']=(X['consumption']-Xstock[0])/Xstock[1]
    Y['washing_machine']=(Y['washing_machine']-Ystock[0][0])/Ystock[0][1]
    Y['fridge_freezer']=(Y['fridge_freezer']-Ystock[1][0])/Ystock[1][1]
    Y['TV']=(Y['TV']-Ystock[2][0])/Ystock[2][1]
    Y['kettle']=(Y['kettle']-Ystock[3][0])/Ystock[3][1]
    return(Xstock,Ystock,X,Y)

In [9]:
def overlap(X_train_data):
    s=X_train_data.shape
    i=509
    j=0
    while (i<s[0]+j):
        temp=X_train_data[i-30:i+1].copy()
        temp2=pd.concat([X_train_data[0:i+1],temp],axis=0)
        X_train_data=pd.concat([temp2,X_train_data[i+1:]],axis=0)
        i=i+510
        j=+30
    return X_train_data

def preprocessing(X,Y):
    traitementallcolumn(X) #reset visibility, temperature, etc
    X_train_data_cleared,Y_train_data_cleared=clean_na(X_train_data, Y_train_data) #clean nan data
    X_train_data_cleared,Y_train_data_cleared=traitement_outliers(X_train_data_cleared, Y_train_data_cleared) #clean outliers
    X_train_data_cleared = overlap(X_train_data_cleared) #duplicate some data not to loose data because of the way the algorithm works
    Xstock, Ystock, X, Y=normalisation(X_train_data_cleared,Y_train_data_cleared) #standardisation
    return(Xstock, Ystock, X, Y)

In [10]:
Xstock,Ystock,X,Y=preprocessing(X_train_data,Y_train_data)
X=X.drop(['visibility','humidity','humidex','windchill','wind','pressure'],axis=1)
Y=Y.drop(['time_step'],axis=1)

[[4.8696637763987605, 44.934154105564495], [49.50874420778702, 52.76751751631555], [14.38747327694926, 17.24908124832397], [2.45964920403363, 72.30863823755402]]  1
[[4.8696637763987605, 44.934154105564495], [49.50874420778702, 52.76751751631555], [14.38747327694926, 17.24908124832397], [2.45964920403363, 72.30863823755402]]  2
[[4.8696637763987605, 44.934154105564495], [49.50874420778702, 52.76751751631555], [14.38747327694926, 17.24908124832397], [2.45964920403363, 72.30863823755402]]  1
